# Multi-instrument analysis with the SunPy Ecosystem

- AIA
- EUI
- SPICE
- DKIST

Bonus:
- IRIS (?)
- EIS (?)

The affiliated packages we will use are:

- `sunpy`
- `sunpy-soar`
- `dkist`
- `sunraster`

The interesting science question we will answer is...

- Observations from chromosphere (dkist) to TR (SPICE) to corona (AIA, EUI)
- How can additional FOV constraints tell us more about what DKIST is seeing?

In [50]:
from sunpy.net import Fido, attrs as attrs
import dkist.net
import sunpy_soar
import astropy.units as u
import solarmach

from sunpy.coordinates import get_earth, get_horizons_coord

ContextualVersionConflict: (matplotlib 3.7.2 (/Users/wtbarnes/mambaforge/envs/spd-2023-sunpy-demo/lib/python3.11/site-packages), Requirement.parse('matplotlib==3.5.3'), {'solarmach'})

## Searching for and Downloading Data with `Fido`

### SDO

In [38]:
time_range = attrs.Time("2022-10-24T19:00", "2022-10-24T19:30")

In [53]:
aia_query = time_range & attrs.Instrument.aia & attrs.Wavelength(171*u.Angstrom) & attrs.Sample(10*u.minute)

In [54]:
q = Fido.search(
    aia_query
)

In [55]:
q

Start Time,End Time,Source,Instrument,Wavelength,Provider,Physobs,Wavetype,Extent Width,Extent Length,Extent Type,Size
,,,,Angstrom,,,,,,,Mibyte
Time,Time,str3,str3,float64[2],str4,str9,str6,str4,str4,str8,float64
2022-10-24 19:00:09.000,2022-10-24 19:00:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844
2022-10-24 19:10:09.000,2022-10-24 19:10:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844
2022-10-24 19:20:09.000,2022-10-24 19:20:10.000,SDO,AIA,171.0 .. 171.0,JSOC,intensity,NARROW,4096,4096,FULLDISK,64.64844


In [57]:
aia_files = Fido.fetch(q, path='data/{instrument}')

Files Downloaded:   0%|                                                                                                                              | 0/3 [00:00<?, ?file/s]
aia_lev1_171a_2022_10_24t19_00_09_34z_image_lev1.fits:   0%|                                                                                     | 0.00/11.8M [00:00<?, ?B/s]


aia_lev1_171a_2022_10_24t19_20_09_35z_image_lev1.fits:   0%|                                                                                     | 0.00/11.8M [00:00<?, ?B/s]


aia_lev1_171a_2022_10_24t19_20_09_35z_image_lev1.fits:   0%|                                                                          | 1.02k/11.8M [00:00<3:00:06, 1.09kB/s]


aia_lev1_171a_2022_10_24t19_20_09_35z_image_lev1.fits:   1%|▌                                                                            | 90.0k/11.8M [00:01<01:41, 116kB/s]


aia_lev1_171a_2022_10_24t19_20_09_35z_image_lev1.fits:   2%|█▍                                                            

### Solar Orbiter

In [42]:
eui_query = time_range & attrs.soar.Product('EUI-HRIEUV174-IMAGE') & attrs.Level(2)
spice_query = time_range & attrs.soar.Product('SPICE-N-RAS') & attrs.Level(2)

In [43]:
q = Fido.search(eui_query | spice_query)

In [44]:
q

Instrument,Data product,Level,Start time,End time,Data item ID,Filename,Filesize,SOOP Name
,,,,,,,Mbyte,
str3,str19,str2,str23,str23,str46,str55,float64,str30
EUI,EUI-HRIEUV174-IMAGE,L2,2022-10-24 19:29:55.174,2022-10-24 19:29:56.824,solo_L2_eui-hrieuv174-image_20221024T192955174,solo_L2_eui-hrieuv174-image_20221024T192955174_V01.fits,5.665,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,EUI-HRIEUV174-IMAGE,L2,2022-10-24 19:29:30.174,2022-10-24 19:29:31.824,solo_L2_eui-hrieuv174-image_20221024T192930174,solo_L2_eui-hrieuv174-image_20221024T192930174_V01.fits,5.679,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,EUI-HRIEUV174-IMAGE,L2,2022-10-24 19:29:10.174,2022-10-24 19:29:11.824,solo_L2_eui-hrieuv174-image_20221024T192910174,solo_L2_eui-hrieuv174-image_20221024T192910174_V01.fits,5.665,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,EUI-HRIEUV174-IMAGE,L2,2022-10-24 19:20:25.173,2022-10-24 19:20:26.823,solo_L2_eui-hrieuv174-image_20221024T192025173,solo_L2_eui-hrieuv174-image_20221024T192025173_V01.fits,5.504,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,EUI-HRIEUV174-IMAGE,L2,2022-10-24 19:28:40.174,2022-10-24 19:28:41.824,solo_L2_eui-hrieuv174-image_20221024T192840174,solo_L2_eui-hrieuv174-image_20221024T192840174_V01.fits,5.662,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,EUI-HRIEUV174-IMAGE,L2,2022-10-24 19:28:20.204,2022-10-24 19:28:21.854,solo_L2_eui-hrieuv174-image_20221024T192820204,solo_L2_eui-hrieuv174-image_20221024T192820204_V01.fits,5.659,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,EUI-HRIEUV174-IMAGE,L2,2022-10-24 19:27:55.188,2022-10-24 19:27:56.838,solo_L2_eui-hrieuv174-image_20221024T192755188,solo_L2_eui-hrieuv174-image_20221024T192755188_V01.fits,5.679,R_SMALL_MRES_MCAD_AR-Long-Term
EUI,EUI-HRIEUV174-IMAGE,L2,2022-10-24 19:27:35.189,2022-10-24 19:27:36.839,solo_L2_eui-hrieuv174-image_20221024T192735189,solo_L2_eui-hrieuv174-image_20221024T192735189_V01.fits,5.665,R_SMALL_MRES_MCAD_AR-Long-Term


### DKIST

In [45]:
dkist_query = time_range & attrs.Instrument('VISP')

In [46]:
Fido.search(dkist_query)

<sunpy.net.fido_factory.UnifiedResponse object at 0x16925f490>
Results from 1 Provider:

0 Results from the DKISTClient:

<No columns>